In [ ]:
import syft as sy
from syft.service.user.user import UserCreate, ServiceRole
import pandas as pd

In [ ]:
server = sy.orchestra.launch(
    name='emory_hospital',
    reset=True,
    port=8094,
    server_side_type='high',
)
dataset_path = 'dataset/subjects_subset_2.csv'

Run the following cell to end the server instance if necessary.

In [ ]:
server.land()

In [ ]:
admin = server.login(email="info@openmined.org", password="changethis")
admin.settings.allow_guest_signup(enable=False)

In [ ]:
admin.users.create(
    name='Gopesh Singal',
    email='gsingal3@gatech.edu',
    password='testing123',
    role=ServiceRole.DATA_SCIENTIST
)

In [ ]:
def create_dataset_assets(path, name, description) -> sy.Asset:
    '''
    Assumption is made the the input path leads to a CSV formatted dataset with substantial rows such that
    100 of the rows can be partitioned into a mock dataset without significantly decreasing the real dataset's 
    size
    '''
    real_df = pd.read_csv(path)

    asset = sy.Asset(
        name=name,
        description=description,
        data=real_df,
        mock=real_df,
    )
    return asset
    
asset_subj = create_dataset_assets(path=dataset_path, name='Subjects', description='Personal information for patients')
dataset = sy.Dataset(
    name='Hospital information', 
    description='Information regarding patients in hospital care for CS 6220',
    asset_list=[asset_subj]
)

admin.upload_dataset(dataset)

The following cell will access the requests from a client process and approve the request. 

In [ ]:
project = admin.projects[0]
request = project.requests[0]
rag_function = request.code.run

result = request.approve()